In [ ]:
from mistralai import Mistral
from tool_config import tools_conf
import json

### IMPORTE NECESSARY FUNCTIONS CALLED IN THE QUERY FUNCTION
# from .... import retrieve_best_productsV0
# from .... import retrieve_best_productsV1
from tools_func import *
# Retrieve the API key from the environment variable
api_key = "W7jZ5RO87zVxhO0gehFjjg0TqyXasmGj"
if not api_key:
    raise ValueError("MISTRAL_API_KEY environment variable not set.")
custom_agent_id = "ag:68495cb5:20241123:expert-tpmc:bbd4e63b"




class Agent:
    def __init__(self,tools_conf = tools_conf,names_to_functions):
        self.products = []
        self.client = Mistral(api_key=api_key)
        self.tools = tools_conf
        self.names_to_functions = names_to_functions
        self.agent_id = custom_agent_id

        self.user_query = ""
        self.data = ""
        self.advice = ""
        self.next_data_query = ""
        self.expert_input = ""
        self.querry_data = ""

    def get_response_to_user(self, user_message):
        self.user_query = user_message
        self.build_expert_input()
        while self.querry_data != "STOP":
            self.update_advice()
            self.get_next_querry()
            self.get_next_action()
            self.update_data(self.querry_data)
        return self.get_reseponse()

        
    def build_expert_input(self):
        self.expert_input = "{\"user_input\": \"" + self.user_query + "\"; \"data_available\": \"" + self.data + "\"; \"Advice\": \"" + self.advice + "\"; \"new_data_query\": \"" + self.next_data_query + "\"}"
        

    def update_advice(self):
        task = "based on the current input, update the advice to be made for the customer. Answer should be in the right format. Only the advice should be updated, and not the other fields"
        txt_message = task + "  " + self.expert_input
        messages = [
            {
                "role": "user",
                "content": txt_message
            }
        ]

        agent_response = self.client.agents.complete(agent_id=self.agent_id, messages=messages)
        self.expert_input = agent_response.choices[0].message.content

    def update_data(self, data):
        task = "The goal is to update the data_available field with new data received through external data. The data_available field should now include this new data, appropriately labeled. For example, if the query suggests products, add the list of products to data_available, labeling it as 'product suggestion'. Instructions: Update data_available with the information from the external data field. Each addition should be labeled with a descriptive tag, based on the external data tag. Be very precise. For example, Suggested products and previously bought products shouldbe labeled differently .Clear the new_data_query field after updating data_available. This field should be set to empty as a result of this operation.Do not modify any other fields besides data_available and new_data_query.Your response should reflect these changes, with data_available updated and new_data_query cleared. Do not remove any information which was the data_available field. "
        txt_message = task + " External data : " + data + "  Current fields :  " + self.expert_input
        messages = [
            {
                "role": "user",
                "content": txt_message
            }
        ]

        agent_response = self.client.agents.complete(agent_id=self.agent_id, messages=messages)
        self.expert_input = agent_response.choices[0].message.content
        
    def get_next_action(self):
        task = "From this inpu state, extract the content of the 'new_data_query' field, and use it to decide what action to take next. The action should be a function call, with the right parameters. The function to call should be chosen based on the new_data_query field. The parameters should be extracted from the new_data_query field. The other fields should be discarded. The action should be in the right format. "


        txt_message = task + " Here is the input state :   " + self.expert_input
        messages = [
            {
                "role": "user",
                "content": txt_message
            }
        ]
    
        
        response = self.client.chat.complete(
                model = "mistral-large-latest",
                messages = messages,
                tools = self.tools,
                tool_choice = "any",
                )
        function_name = response.choices[0].message.tool_calls[0].function.name
        function_params = json.loads(response.choices[0].message.tool_calls[0].function.arguments)
        print(function_name)
        print(function_params)
        self.querry_data = self.names_to_functions[function_name](**function_params)
        
    
    def get_next_querry(self):
        task = "based on the client input, and the available data, get the next data query to be made. Use actions, verbs, to indicate what should be querried, such as Find... Indentify... . If we already have the needed information, the data querry can be empty, 'no more information needed'. Take into account all the available data and advcie. Dont ask more than needed. You should stop at some point. Only look for information that you don't already have. When possible, try to look for products that might be good for the user, if none have been identified yet. The query should contain all the necessary information for the querry agent to choose the right function to call and give the right argument. Be precise. Only lookf for one information at a time. Only modify the querry data field.  Answer should be in the right format. "
        txt_message = task + "  " + self.expert_input
        messages = [
            {
                "role": "user",
                "content": txt_message
            }
        ]

        agent_response = self.client.agents.complete(agent_id=self.agent_id, messages=messages)
        self.expert_input = agent_response.choices[0].message.content

    def get_reseponse(self):
        task = "It seems all the information that could be obtained from the data base is in your possessing. Based on the current input, get the response to be made to the customer. Answer should be a string understandable by the client. Data querry should now be empty, as the information is transfered from there to available data. "
        txt_message = task + "  " + self.expert_input
        messages = [
            {
                "role": "user",
                "content": txt_message
            }
        ]

        agent_response = self.client.agents.complete(agent_id=self.agent_id, messages=messages)
        self.expert_input = agent_response.choices[0].message.content
        return self.expert_input

In [168]:


tools_conf = [
    {
        "type": "function",
        "function": {
            "name" : "no_more_info_needed",
            "description": "If no more information is needed, call this function, to stop the process.",
            "parameters": {}
            }
    },
    {
        "type": "function",
        "function": {
            "name": "get_k_nearests_product",
            "description": "Search for the k products closest to the user's needs. Here the goal is to find products to be suggested to the client.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The query string to search for similar products."
                    },
                    "k": {
                        "type": "integer",
                        "description": "The number of nearest products to retrieve."
                    }
                },
                "required": ["query"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_k_nearest_users",
            "description": "Search for users who have a similar profile to the target user.",
            "parameters": {
                "type": "object",
                "properties": {
                    "user_id": {
                        "type": "integer",
                        "description": "The unique identifier for the user."
                    },
                    "k": {
                        "type": "integer",
                        "description": "The number of nearest users to retrieve."
                    }
                },
                "required": ["user_id", "k"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_best_purchases_from_neighbours",
            "description": "Retrieve the best products purchased by other users which have with similar profiles. Not related to the current user's previous purchaes.",
            "parameters": {
                "type": "object",
                "properties": {
                    "user_id": {
                        "type": "integer",
                        "description": "The unique identifier for the user."
                    }
                },
                "required": ["user_id"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_not_delivered",
            "description": "Returns a text summarizing the undelivered purchases of the user.",
            "parameters": {
                "type": "object",
                "properties": {
                    "user_id": {
                        "type": "integer",
                        "description": "The unique identifier for the user."
                    }
                },
                "required": ["user_id"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "handle_insufficient_info",
            "description": "Handles cases where there is not enough information to proceed with a product search and requests additional details from the user.",
            "parameters": {
                "type": "object",
                "properties": {
                    "missing_fields": {
                        "type": "array",
                        "items": {
                            "type": "string"
                        },
                        "description": "The specific fields that are missing from the input, requiring user input."
                    },
                    "suggested_fields": {
                        "type": "array",
                        "items": {
                            "type": "string"
                        },
                        "description": "Fields that are most critical to gather additional information for a search."
                    },
                    "action": {
                        "type": "string",
                        "enum": ["request_info", "abort"],
                        "description": "Action to be taken: request more information or abort the process."
                    }
                },
                "required": ["missing_fields", "action"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "check_user_purchase_history",
            "description": "Analyzes user purchase history and current orders to identify relevant patterns or preferences for product recommendations. Also provides all previous orders when the user previous orders is requested.",
            "parameters": {
                "type": "object",
                "properties": {
                    "user_id": {
                        "type": "integer",
                        "description": "The unique identifier for the user whose purchase history is being analyzed."
                    },
                    "k": {
                        "type": "integer",
                        "description": "How many purchases to recover."
                    }
                },
                "required": ["user_id", "k"],
                "additionalProperties": False
            }
        }
    }
]


In [169]:
# fichier des function appellé par query 
from User.db_utilitaries import RetrieveDatabase, UpdateDatabase
from RAG_product.Faiss_database import FaissDatabase
from Product.utils_db_product import Retrieve_from_db_prd
from basemistral import BaseMistral

from typing import Tuple

def no_more_info_needed():
    return "STOP"

def get_k_purchase(user_id:int, k:int = 5)-> str:
    """recupère les k dernière purchases dans la database

    Args:
        user_id (int): 
        k (int, optional):  Defaults to 5.

    Returns:
        str
        """
    user_id = 0 #set pour la demonstration
    res = ""
    purchases = RetrieveDatabase.get_last_k_purchases_by_user_id(user_id, k )
    for purchase in purchases:
        res += RetrieveDatabase.tuple_to_detailed_text(purchase)
        res += '\n' 
    if not res:
        res = "NO HISTORY"
    return res 



def get_k_nearests_product(query: str, k :int = 3) -> str :
    """Use RAG to get the 5 nearest product from the query , based on similarity with the descriptions 

    Args:
        query (str): 
        k (int, optional): Defaults to 3.
    """
    id_produits_list = FaissDatabase.search(query, k)
    produits = Retrieve_from_db_prd.get_product_by_id_list(id_produits_list)
    products_refined = [(produit[1], produit[2], produit[4]) for produit in produits]
    
    
    #Appeler la fonction qui affiche les produits
    
    #on va résumer les infos, donc pour ca on appelle baseMistral
    model = BaseMistral()
    text = ''
    i=0
    for produit in produits:
        text_produit = Retrieve_from_db_prd.product_to_paragraph(produit)
        text += f"Produit {i}: \n "
        text += model.summarize(text_produit)
        text += '\n'
        i+=1
        
    return text, products_refined



def get_k_nearest_users( user_id : int , k= 3) -> list[Tuple]:
    """ utilise faiss pour rechercher les k users les plus proches en ce basant sur le texte d'information

    Args:
        user_id (int)

    Returns:
        list[Tuple]
    """
    path_db_users = "Manager/User/userDB/user_index_faiss.faiss"
    user_id = 0 # pour le test
    
    user_info = RetrieveDatabase.get_user_info(user_id)
    
    index_neighbours = FaissDatabase.search(user_info, k, db_path= path_db_users)
    
    users = []
    for i in range(len(index_neighbours)):
        if index_neighbours[i] == user_id:
            index_neighbours.pop(i)
        else:
            users.append(RetrieveDatabase.get_user_by_id(index_neighbours[i]))
    return index_neighbours, users 


def get_best_purchases_from_neighbours(user_id = 0) -> str:
    """ Renvoie les 3 meilleurs achats des  3 voisins les plus proches, en version texte , cf function 

    Args:
        user_id (int, optional): _description_. Defaults to 0.
    """
    user_id = 0 
    index_neighbours, _  = get_k_nearest_users(user_id, 3)
    purchases =  []
    for i in index_neighbours:
        neighbours_id = index_neighbours[i]
        purchase_neighbours = RetrieveDatabase.get_best_k_purchases_by_user_id(neighbours_id, 3)
        purchases.extend(purchase_neighbours)
    text = ""
    for purchase in purchases : 
        text += RetrieveDatabase.tuple_to_detailed_text(purchase)
        text += "\n"
        
    return text 



def get_not_delivered(user_id: int) -> str:
    """renvoie un texte résumant les purchases non délivrer du user

    Args:
        user_id (int)
    """
    purchases = RetrieveDatabase.get_not_delivered_purchases(user_id)
    text = "Here are the commands not received: \n"
    
    for purchase in purchases:
        product_name, purchase_date  = purchase[2], purchase[3]
        text += f"{product_name}, purchased on the {product_name}"
        
    return text



def cancel_purchase(user_id: int,
                      purchase_id: int = None,
                      product_name: str = None,
                      purchased_date = None) -> str:
    """cancel order

    Args:
        user_id (int): 
        purchase_id (int, optional):. Defaults to None.
        product_name (str, optional): Defaults to None.
        purchased_date (_type_, optional):  Defaults to None.

    Returns:
        str: _description_
    """
    res = UpdateDatabase.suppress_purchase(user_id, purchase_id, product_name, purchased_date)
    return res 

names_to_functions = {
    "get_k_purchase": get_k_purchase,
    "get_k_nearests_product": get_k_nearests_product,
    "get_k_nearest_users": get_k_nearest_users,
    "get_best_purchases_from_neighbours": get_best_purchases_from_neighbours,
    "get_not_delivered": get_not_delivered,
    "cancel_purchase": cancel_purchase,
    "no_more_info_needed": no_more_info_needed
}



    

    

In [170]:
agent = Agent(tools_conf=tools_conf, names_to_functions=names_to_functions)


In [171]:
agent.user_query = "I have dry skin. I also want my ;ast purchases"
agent.build_expert_input()
print(agent.expert_input)
agent.update_advice()
print(agent.expert_input)
agent.get_next_querry()
print(agent.expert_input)

{"user_input": "I have dry skin. I also want my ;ast purchases"; "data_available": ""; "Advice": ""; "new_data_query": ""}
{"user_input": "I have dry skin. I also want my ;ast purchases", "data_available": "", "Advice": "For dry skin, focus on hydrating and moisturizing products. Look for ingredients like hyaluronic acid, glycerin, and ceramides. Also, consider using gentle cleansers and avoiding products with alcohol. I will retrieve your last purchases from our records.", "new_data_query": ""}
{"user_input": "I have dry skin. I also want my ;ast purchases", "data_available": "", "Advice": "For dry skin, focus on hydrating and moisturizing products. Look for ingredients like hyaluronic acid, glycerin, and ceramides. Also, consider using gentle cleansers and avoiding products with alcohol. I will retrieve your last purchases from our records.", "new_data_query": "Find last purchases made by the user"}


In [160]:
print(agent.expert_input)

```json
{"user_input": "I have dry skin. I also want my last purchases."; "data_available": ""; "Advice": "To address your dry skin, consider using a rich moisturizer with ingredients like hyaluronic acid and ceramides. For your last purchases, please provide the specific products or brands you're interested in."; "new_data_query": "Identify the client's last purchases."}
```


In [174]:
agent.get_next_action()


check_user_purchase_history
{'k': 5, 'user_id': 1}


KeyError: 'check_user_purchase_history'

In [117]:
print(agent.querry_data)

("Produit 0: \n **Summary:**\n\nThe '5 Voskhod Teflon Coated Double Edge Razor Blades' are priced at $5.49 and have an average rating of 4.0 out of 5. Imported from Russia, these blades feature a Teflon coating and are designed for double-edge safety razors.\nProduit 1: \n The 'ANOGOL Hair Cap+Lace Front Platinum Blonde Wig' is a synthetic hair wig priced at $46.99, with an average rating of 3.0 out of 5 based on numerous reviews. This wig features a long, curly blonde lace front, designed for women's daily life, movie cosplay, dress-up, and parties. The wig is 24 inches long with a circumference of about 22.5 inches, making it an average size. It includes 180% density, 3 combs inside, and 2 hooks. The wig care involves spraying wig care solution 2-3 times, keeping the wig off the solution for 15-25cm when spraying, and combing the wig gently from the ends. Instructions on how to wear the wig are also provided, detailing steps to tie up hair, stretch the wig cap, push hair into the cap

In [179]:
agent.update_data(agent.querry_data[0])    
print(agent.expert_input)

```json
{
  "user_input": "I have dry skin. I also want my ;ast purchases",
  "data_available": "Previous purchases : Product 1 - 10 €. Product Suggestions : Produit 0: 'Zucker Feather (TM) - Marabou Antenna Headband w/Lurex Dark Lilac/Opal Lurex' is priced at $7.40 and has an average rating of 2.0 out of 5. The product is a headband with Lurex in dark lilac and opal colors. Produit 1:  The 'Got 2B Kinkier Curling Mousse 9.6oz Bonus' is priced at $75.00 and has a perfect rating of 5.0 out of 5 based on [number of reviews unspecified] reviews. Produit 2:  The 'NYC New York Color HD Waterproof Automatic Eyeliner - #004 Turquoise' is priced at $9.99 and has an average rating of 1.0 out of 5. The product is designed to make applying eyeliner easy and swift, with a pen-like twist mechanism that reveals a precise color point. It also includes a brush-style blending tool for customizing your look.",
  "Advice": "For dry skin, focus on hydrating and moisturizing products. Look for ingredients 

In [176]:
agent.update_data("Previous pruchases : PRoduct 1 - 10 €")    
print(agent.expert_input)

```json
{
  "user_input": "I have dry skin. I also want my ;ast purchases",
  "data_available": "Previous purchases : Product 1 - 10 €",
  "Advice": "For dry skin, focus on hydrating and moisturizing products. Look for ingredients like hyaluronic acid, glycerin, and ceramides. Also, consider using gentle cleansers and avoiding products with alcohol. I will retrieve your last purchases from our records.",
  "new_data_query": ""
}
```


In [103]:
agent.update_data("cleanser 1 : 10€")

In [104]:
print(agent.expert_input)

```json
{
  "user_query": "Client has dry skin.",
  "data_available": "Client has dry skin. Products: cleanser 1: 10€",
  "Advice": "Consider using a gentle, hydrating cleanser to maintain skin moisture.",
  "data_query": "Hydrating cleansers for dry skin"
}
```


In [181]:
agent.get_next_querry()
print(agent.expert_input)

```json
{
  "user_input": "I have dry skin. I also want my ;ast purchases",
  "data_available": "Previous purchases : Product 1 - 10 €. Product Suggestions : Produit 0: 'Zucker Feather (TM) - Marabou Antenna Headband w/Lurex Dark Lilac/Opal Lurex' is priced at $7.40 and has an average rating of 2.0 out of 5. The product is a headband with Lurex in dark lilac and opal colors. Produit 1:  The 'Got 2B Kinkier Curling Mousse 9.6oz Bonus' is priced at $75.00 and has a perfect rating of 5.0 out of 5 based on [number of reviews unspecified] reviews. Produit 2:  The 'NYC New York Color HD Waterproof Automatic Eyeliner - #004 Turquoise' is priced at $9.99 and has an average rating of 1.0 out of 5. The product is designed to make applying eyeliner easy and swift, with a pen-like twist mechanism that reveals a precise color point. It also includes a brush-style blending tool for customizing your look.",
  "Advice": "For dry skin, focus on hydrating and moisturizing products. Look for ingredients 

In [182]:
agent.get_next_action()

no_more_info_needed
{}


In [96]:
agent.querry_data

'STOP'

In [183]:
print(agent.get_reseponse())

Based on your input, here's my response:

"For your dry skin, I recommend focusing on hydrating and moisturizing products. Look for ingredients like hyaluronic acid, glycerin, and ceramides. Also, consider using gentle cleansers and avoiding products with alcohol.

Your last purchase was:
- Product 1 - 10 €.

Additionally, here are some product suggestions:
1. 'Zucker Feather (TM) - Marabou Antenna Headband w/Lurex Dark Lilac/Opal Lurex' priced at $7.40 with an average rating of 2.0 out of 5.
2. 'Got 2B Kinkier Curling Mousse 9.6oz Bonus' priced at $75.00 with a perfect rating of 5.0 out of 5.
3. 'NYC New York Color HD Waterproof Automatic Eyeliner - #004 Turquoise' priced at $9.99 with an average rating of 1.0 out of 5.

Would you like more recommendations or have any other questions?"
